In [ ]:
! nvidia-smi

Thu Jan 11 01:31:24 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 데이터 준비

In [ ]:
# 출처: https://github.com/e9t/nsmc
import pandas as pd
df_train = pd.read_csv('ratings_train.txt', sep='\t')
df_test = pd.read_csv('ratings_test.txt', sep='\t')

df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149995 entries, 0 to 149994
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49997 entries, 0 to 49996
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


## 전처리 (토큰화)

In [ ]:
!pip install tensorflow_text
# !pip install tensorflow_hub

import tensorflow_text as text

import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

%load_ext tensorboard
tf.__version__

In [ ]:
preprocessor = hub.load("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/multi-cased-preprocess/versions/3")

text_inputs = [tf.keras.layers.Input(shape=(), dtype=tf.string)]
tokenize = hub.KerasLayer(preprocessor.tokenize)
tokenized_inputs = [tokenize(segment) for segment in text_inputs]

seq_length = 32  # 선택
bert_pack_inputs = hub.KerasLayer(
    preprocessor.bert_pack_inputs,
    arguments=dict(seq_length=seq_length))
encoder_inputs = bert_pack_inputs(tokenized_inputs)

preprocessed_text = tf.keras.Model(text_inputs, encoder_inputs)
preprocessed_text.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None,)]                 0         
                                                                 
 keras_layer (KerasLayer)    (None, None, None)        0         
                                                                 
 keras_layer_1 (KerasLayer)  {'input_mask': (None, 3   0         
                             2),                                 
                              'input_word_ids': (Non             
                             e, 32),                             
                              'input_type_ids': (Non             
                             e, 32)}                             
                                                                 
Total params: 0 (0.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_______________________________

In [ ]:
print(df_test['document'][2])
preprocessed_text(tf.constant([df_test['document'][2]]))

뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아


{'input_mask': <tf.Tensor: shape=(1, 32), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 32), dtype=int32, numpy=
 array([[   101,   9303,  21711,   9638,   9926,  34907,  22879,    119,
            119,    119,    119,   8982, 119022,  18623,   9523,  28578,
          10150,  34907,   9715,  26344,   9074,  54141, 119168,    100,
            102,      0,      0,      0,      0,      0,      0,      0]],
       dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(1, 32), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}

## 모델링

In [ ]:
model_url = "https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4"

encoder = hub.KerasLayer(model_url, trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]
output = tf.keras.layers.Dense(2, activation='softmax')(pooled_output)
model = tf.keras.Model(text_inputs, output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    (None, None, None)           0         ['input_1[0][0]']             
                                                                                                  
 keras_layer_1 (KerasLayer)  {'input_mask': (None, 32),   0         ['keras_layer[0][0]']         
                              'input_word_ids': (None,                                            
                             32),                                                                 
                              'input_type_ids': (None,                                      

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=5e-5),
              loss='sparse_categorical_crossentropy',
              metrics='accuracy'
              )

In [ ]:
model.fit(df_train['document'], df_train['label'],
          validation_data=(df_test['document'], df_test['label']), epochs=1, batch_size=64)

2344/2344 [==============================] - 1729s 721ms/step - loss: 0.4106 - accuracy: 0.8067 - val_loss: 0.3566 - val_accuracy: 0.8397


## 실행


In [ ]:
import tensorflow_text as text

import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime
from google.colab import drive

import pandas as pd
from tqdm import tqdm

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import joblib

# ## 모델 저장할 때
# joblib.dump(model, 'model.pkl')

# ## 불러올 때
# model2 = joblib.load("model.pkl")



# model.save('/content/drive/MyDrive/ML_TFT/model.pkl')

In [ ]:
model2 = tf.saved_model.load('/content/drive/MyDrive/ML_TFT/model.pkl')

In [ ]:
model.predict(['영화가 재미 있어요'])

1/1 [==============================] - 0s 39ms/step


array([[0.04866978, 0.9513302 ]], dtype=float32)

In [ ]:
model2(['영화가 재미 있어요'])

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0.04866978, 0.9513302 ]], dtype=float32)>

In [ ]:
model.predict(['영화가 재미 없어요'])

1/1 [==============================] - 0s 38ms/step


array([[0.98152924, 0.01847081]], dtype=float32)

In [ ]:
tensor = model2(['영화가 재미 d없어요'])
numpy_array = tensor.numpy()
print(numpy_array)

[[0.96076334 0.03923673]]


In [ ]:
label = np.argmax(numpy_array, axis=1)[0]
label


0

In [ ]:

df = pd.read_csv('/content/drive/MyDrive/ML_TFT/Youtube_Comment.csv')
df.head()

,TITLE_NM,CONTENT
0,추천 꿀조합 후추치즈 열라면 #열라면 #순후추,열라면은 순두부랑도 어울리고 치즈랑도 어울리고 ㄹㅇ 만능라면이네
1,추천 꿀조합 후추치즈 열라면 #열라면 #순후추,어으 나는 치즈가 라면국물에 섞이는 게 싫던데...
2,추천 꿀조합 후추치즈 열라면 #열라면 #순후추,오늘 마열라면에 치즈 두장 넣어 먹었는데 크으
3,[오뚜기 마열라면] 한입만 더!를 계속 외칠 수밖에 없던 마열라면의 매력❤️,맛없으면 반역 맛있으면 혁명 아닙니까 !
4,[오뚜기 마열라면] 한입만 더!를 계속 외칠 수밖에 없던 마열라면의 매력❤️,입에 넣어준다는 대사가 저런 뜻이었구나


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21725 entries, 0 to 21724
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   TITLE_NM  21725 non-null  object
 1   CONTENT   21652 non-null  object
dtypes: object(2)
memory usage: 339.6+ KB


In [ ]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21652 entries, 0 to 21651
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   TITLE_NM  21652 non-null  object
 1   CONTENT   21652 non-null  object
dtypes: object(2)
memory usage: 338.4+ KB


In [ ]:
def get_label(content):
    tensor = model2([content])
    numpy_array = tensor.numpy()
    return np.argmax(numpy_array, axis=1)[0]

In [ ]:
import gc


# 배치 크기
batch_size = 128

results = []

for i in tqdm(range(0, len(df), batch_size)):

    batch = df['CONTENT'][i:i+batch_size].tolist()
    tensor = model2(batch)
    numpy_array = tensor.numpy()

    batch_results = np.argmax(numpy_array, axis=1).tolist()
    results.extend(batch_results)

    del batch, tensor, numpy_array, batch_results

    # 가비지 컬렉터 실행
    gc.collect()

100%|██████████| 170/170 [06:14<00:00,  2.20s/it]


In [ ]:
df['label'] = results

In [ ]:
df.head(20)

,TITLE_NM,CONTENT,label
0,추천 꿀조합 후추치즈 열라면 #열라면 #순후추,열라면은 순두부랑도 어울리고 치즈랑도 어울리고 ㄹㅇ 만능라면이네,1
1,추천 꿀조합 후추치즈 열라면 #열라면 #순후추,어으 나는 치즈가 라면국물에 섞이는 게 싫던데...,0
2,추천 꿀조합 후추치즈 열라면 #열라면 #순후추,오늘 마열라면에 치즈 두장 넣어 먹었는데 크으,1
3,[오뚜기 마열라면] 한입만 더!를 계속 외칠 수밖에 없던 마열라면의 매력❤️,맛없으면 반역 맛있으면 혁명 아닙니까 !,1
4,[오뚜기 마열라면] 한입만 더!를 계속 외칠 수밖에 없던 마열라면의 매력❤️,입에 넣어준다는 대사가 저런 뜻이었구나,0
5,[오뚜기 마열라면] 한입만 더!를 계속 외칠 수밖에 없던 마열라면의 매력❤️,황두환,1
6,[오뚜기 맛있는 눈꽃교자] 맛있는 눈꽃이~ 피었습니다❄️🥟,Oh! Deve ser uma delicia! Quero provar. Será q...,0
7,[오뚜기 맛있는 눈꽃교자] 맛있는 눈꽃이~ 피었습니다❄️🥟,와우 김태희 참 이쁘다,1
8,[오뚜기 맛있는 눈꽃교자] 맛있는 눈꽃이~ 피었습니다❄️🥟,이자,0
9,[오뚜기 맛있는 눈꽃교자] 맛있는 눈꽃이~ 피었습니다❄️🥟,오 맛있겠당,1


In [ ]:
df.to_csv("/content/drive/MyDrive/ML_TFT/labeled_df.csv", index=False, encoding="utf-8-sig")

In [ ]:
def run_bert_cls():
    print('==================== BERT 문장 긍부정 예측 ====================')
    while True:
        sentence =input('문장을 입력해주세요: ')
        if sentence == '0':
            break
        pred = model2([sentence])[0]
        negative = pred[0] * 100
        positive = pred[1] * 100
        print('긍정일 확률: {:.2f}%'.format(positive))
        print('부정일 확률: {:.2f}%'.format(negative))

In [ ]:
run_bert_cls()